In [2]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.get(name="quick-starts-ws-125912")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125912
Azure region: southcentralus
Subscription id: 4910dccd-0348-46c4-a51f-d8c85e078b14
Resource group: aml-quickstarts-125912


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azureml.core.compute_target import ComputeTargetException

compute_name = "mycompute"
try:
    compute = ComputeTarget(workspace=ws, name=compute_name)
    print('Compute cluster {} already exists!'.format(compute_name))
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute = ComputeTarget.create(ws, compute_name, config)
    
compute.wait_for_completion()

In [27]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
from azureml.train.hyperdrive import loguniform, choice
ps = RandomParameterSampling(
    {
        '--C': loguniform(-5, 3), 
        '--max_iter': choice(50, 100, 500, 1000, 5000, 10000)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=100, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
import shutil
training_folder = "./training"
shutil.copyfile('train.py', os.path.join(training_folder, 'train.py'))

# Create a SKLearn estimator for use with train.py
from azureml.train.estimator import Estimator
est = Estimator(source_directory=training_folder,
                entry_script="train.py",
                compute_target=compute,
                conda_packages=['scikit-learn==0.21.3', 'pandas==0.23.4']
               )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
#    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
#    max_total_runs=1000)
    max_total_runs=15)

In [28]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(hyperdrive_config)
from azureml.widgets import RunDetails
RunDetails(run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
model = best_run.register_model(model_name='lr_bankmarketing', model_path='outputs/lr_bankmarketing.joblib')


AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
from train import clean_data
x, y = clean_data(dataset)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=x.join(y),
    label_column_name="y",
    n_cross_validations=5)


In [ ]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=False)

In [ ]:
# Retrieve and save your best automl model.
automl_run.get_output()
best_automl_run =automl_run.get_best_run_by_primary_metric()
print(best_automl_run.get_metrics())
model = best_automl_run.register_model(model_name='automl_bankmarketing', model_path='outputs/automl_bankmarketing.joblib')

In [ ]:
# Clean up compute cluster
try:
    compute.delete()
except ComputeTargetException as e:
    print(e.exception_message)
    print("Failed to clean up compute cluster!)
          
try:
    ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    print('Compute cluster {} no longer exists.'.format(compute_name))